## Make and view annual time-series data as 3-band composite

In [ ]:
import os
import sys
from pathlib import Path
import datetime

import rasterio as rio
from rasterio import plot
from rasterio.plot import show
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from ts_composite import *
from plot import *
from ts_profile import *

In [ ]:
'''
PARAMETERS: modify basicConfig in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n PrintDate = {} \n gridCell = {} \n index_dir = {} \n out_dir = {} \n spec_index= {} \n filter_yr = {}"
      .format(basic_config['today'],basic_config['grid_cell'],basic_config['index_dir'],basic_config['local_dir'],basic_config['spec_index'],basic_config['filter_yr']))
%store -r plot_params

if basic_config['ptfile'] == 'interactive':
    print('Using interactive point file, with coordinates stored at {}/SelectedCoords.csv'.format(basic_config['local_dir']))
else:
    print( "Shapefile = {} \n    If point, file is {} \n    If poly, file is {} \n input crs = {}"
     .format (plot_params['shpfile'], basic_config['ptfile'],basic_config['polyfile'],plot_params['inputCRS']))

%store -r timeseries_params
print( "startYr = {} \n endYr = {} \n start_mo = {} \n"
     .format (timeseries_params['start_yr'],timeseries_params['end_yr'],timeseries_params['start_mo']))

In [ ]:
if plot_params['interactive'] == True:
    from ipywidgets import Label
    from ipyleaflet  import Map, GeoData, basemaps, LayersControl, ImageOverlay, Marker, Popup
    from localtileserver import get_leaflet_tile_layer, TileClient
    from plot_interactive import *

## Current options for composite bands are:
* Avg (mean)
* Max (maximum index value)
* Min (minumum index value)
* Amp (Max - Min)
* Std (standard deviation)
* CV (coefficient of variation: std/mean)
* MaxDate (day of year on which maximum occurs)
* MaxDateCos (cosine of max date, to reduce periodic effect)
* MinDate (day of year on which minimum occurs)
* MinDateCos (cosine of min date, to reduce periodic effect)
---------------------------------------------------------------
* Jan (value from Jan 10th image)
* Apr (value from Apr 10th image)
* Jun (value from Jun 10th image)
* Aug (value from Aug 10th image)
* Nov (value from Nov 10th image)

### note: if running many these, use bash script "MakeTScomposite.sh" to run on job node of cluster

## Make ['Min','Max','Amp'] composite to highlight crops vs grassland/forest

In [ ]:
si_vars = ['maxv_yr','minv_yr','amp_yr']
#si_vars = ['Jan_20','Jun_20','Nov_20']

comp_dir = os.path.join(basic_config['smooth_dir'],'{:06d}'.format(basic_config['grid_cell']),'comp')
comp_path = os.path.join(comp_dir,'{:06d}_{}_{}_{}{}{}.tif'.format(basic_config['grid_cell'],basic_config['filter_yr'],
                                                                   basic_config['spec_index'],si_vars[0],si_vars[1],si_vars[2]))
if Path(comp_path).exists():
    print('comp already exists')
    comp = comp_path
else:
    comp = make_ts_composite(basic_config['grid_cell'],basic_config['index_dir'],comp_dir,basic_config['filter_yr'],
                             timeseries_params['start_mo'],basic_config['spec_index'],si_vars)
    
if type(comp) is str:
    comp = comp
else:
    comp = comp[0]

## Non-interactive image with enhanced colors:

In [ ]:
rgb = get_rbg_img(comp,1)
fig = plt.figure(figsize=(18,12))
plt.imshow(rgb);

## Interactive image (to query points for timeseries profile)

In [ ]:
tile_client = TileClient(comp,port=5554)
m = Map(center=tile_client.center(), zoom=12, basemap=basemaps.Esri.WorldImagery)
t = get_leaflet_tile_layer(tile_client, band=[3,2,1])
m.add_layer(t)

def get_coords(**kwargs):
    if kwargs.get('type') == 'click':
        label = Label()
        label.value = str(kwargs.get('coordinates'))
        #m.add_layer(Marker(location=kwargs.get('coordinates')))
        coords =eval(label.value) 
        selected_coords.append(coords)
        print(selected_coords)
        return selected_coords
selected_coords = []
m.on_interaction(get_coords)

print('click on map to get coordinates to query (can zoom in first)')
m

In [ ]:
coord_dir = Path(basic_config['local_dir']/'coords').mkdir(parents=True, exist_ok=True)
coord_dir = Path(basic_config['local_dir']/'coords')
if plot_params['get_new_coords'] == True:
    coords = convert_and_print_coord_list(selected_coords, img_crs=plot_params['inputCRS'], out_dir=coord_dir)

coords = pd.read_csv(os.path.join(coord_dir,'SelectedCoords.csv'))
    
vals = get_values_at_coords(coords, plot_params['inputCRS'], comp, [1,2,3])
print(coords)

for index, pt in coords.iterrows():
    marker = Marker(location = [pt.lon,pt.lat])
    popup = Popup(
        location=[pt.lon,pt.lat],
        close_button=False,
        auto_close=False,
        close_on_click=False,
        close_on_escape_key=False
    )
    popup.children = "{}: {}/{}/{}".format(index+1,vals[1][index],vals[2][index],vals[3][index])
    print(popup.children)
    message=Label()
    message.value = popup.children
    marker.popup = message
    m.add_layer(marker)

display(m)

## Get time series for pts

In [ ]:
out_dir = os.path.join(basic_config['local_dir']/'TSdfs')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
coords = os.path.join(basic_config['local_dir'],'coords','SelectedCoords.csv')

ts_smooth = get_timeseries_for_pts_multicell(out_dir, basic_config['spec_index'], timeseries_params['start_yr'], timeseries_params['end_yr'],
            basic_config['smooth_dir'], 'Smooth', basic_config['grid_file'], [basic_config['grid_cell']], 
            polyfile=basic_config['polyfile'], oldest=timeseries_params['oldest_samp'], newest=timeseries_params['newest_samp'],
            npts=timeseries_params['npts'], seed=timeseries_params['seed1'], load_samp=True, ptfile=coords)

ts_raw =   get_timeseries_for_pts_multicell(out_dir, basic_config['spec_index'], timeseries_params['start_yr'], timeseries_params['end_yr'],
            basic_config['raw_dir'], 'AllRaw', basic_config['grid_file'], [basic_config['grid_cell']], 
             polyfile=basic_config['polyfile'], oldest=timeseries_params['oldest_samp'], newest=timeseries_params['newest_samp'],
            npts=timeseries_params['npts'], seed=timeseries_params['seed1'], load_samp=True, ptfile=coords)

## show output

In [ ]:
display(m)
fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(15, 8), layout="constrained")
axs[0,0].errorbar(ts_raw.index, ts_raw[0], fmt='o', color='r')                
axs[0,0].errorbar(ts_smooth.index, ts_smooth[0], fmt='o', color='k')
axs[0,0].set_ylim(0,8000)
axs[0,0].set_title('point 1')
if 1 in ts_smooth.columns:
    axs[0,1].errorbar(ts_raw.index, ts_raw[1], fmt='o', color='r')                      
    axs[0,1].errorbar(ts_smooth.index, ts_smooth[1], fmt='o', color='k')
    axs[0,1].set_ylim(0,8000)
    axs[0,1].set_title('point 2')
if 2 in ts_smooth.columns:
    axs[1,0].errorbar(ts_raw.index, ts_raw[2], fmt='o', color='r')                        
    axs[1,0].errorbar(ts_smooth.index, ts_smooth[2], fmt='o', color='k')
    axs[1,0].set_ylim(0,8000)
    axs[1,0].set_title('point 3')
if 3 in ts_smooth.columns:
    axs[1,1].errorbar(ts_raw.index, ts_raw[3], fmt='o', color='r')                        
    axs[1,1].errorbar(ts_smooth.index, ts_smooth[3], fmt='o', color='k')
    axs[1,1].set_ylim(0,8000)
    axs[1,1].set_title('point 4')
fig.suptitle('Smoothed vs Raw {} for {}-{} for sampled coordinates'.format(basic_config['spec_index'],timeseries_params['start_yr'],timeseries_params['end_yr']));

if saving this notebook as static HTML, 
need to make an JPEG of the interactive composite with point markers above and post it here 
(interactive maps from localtileserver will not render with nbconvert currently)
delete the image after rendering and reinsert this text

## to show one point in more detiail:

In [ ]:
fig = plt.figure(figsize=(10, 5))
plt.errorbar(ts_raw.index, ts_raw[0], fmt='o', color='g')
plt.errorbar(ts_smooth.index, ts_smooth[0], fmt='o', color='k')
plt.title('Smoothed vs Raw {} for {}-{} for sampled coordinate'.format(basic_config['spec_index'],timeseries_params['start_yr'],timeseries_params['end_yr']))
#plt.ylim(0,7000)
plt.show();

## Get phenology variables

In [ ]:
season = 'wet'
if season == 'wet':
    phen_bands = ['SOS_wet','EOS_wet','LOS_wet','ROG_wet','ROS_wet']
    phen_comp = os.path.join(comp_dir,'{:06d}'.format(int(basic_config['grid_cell']))+'_'+str(basic_config['filter_yr'])+basic_config['spec_index']+'_'+'PhenWet.tif')
    if Path(phen_comp).exists():
        print('comp already exists')
    else:
        phen_comp = make_ts_composite(basic_config['grid_cell'], basic_config['index_dir'],comp_dir,basic_config['filter_yr'],timeseries_params['start_mo'],basic_config['spec_index'],phen_bands)

In [ ]:
phen_vals = get_values_at_coords(coords, plot_params['inputCRS'], phen_comp, [1,2,3,4,5])
print(phen_vals)

## Make ['Jan','Jun','Nov'] composite to see different crops

In [ ]:
bands = ['Jan','Jun','Nov']
band_name = '{}{}{}.tif'.format(bands[0],bands[1],bands[2])
comp_dir = os.path.join(basic_config['smooth_dir'],'{:06d}'.format(basic_config['grid_cell']),'comp')
comp2 = ('{}/{:06d}'.format(comp_dir, basic_config['grid_cell'])+'_'+str(basic_config['filter_yr'])+basic_config['spec_index']+'_'+band_name+'2')
if Path(comp2).exists():
    print('comp already exists')
else:
    comp2 = make_ts_composite(basic_config['grid_cell'],basic_config['index_dir'],comp_dir,basic_config['filter_yr'],basic_config['spec_index'],bands)

In [ ]:
rgb = get_rbg_img(comp2,1)
fig = plt.figure(figsize=(18,12))
plt.imshow(rgb);

# To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html

out_name = str(basic_config['country']+'2b_ViewTimeSeriesComposite'+'_cell'+str(basic_config['grid_cell'])+'_'+str(basic_config['filter_yr']))
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --ExecutePreprocessor.store_widget_state=True --output=$out_name 2b_ViewTimeSeriesComposite.ipynb